In [ ]:
import pygad
import os
import json

In [ ]:
def make_example_dict(values):
    reward_dict = {}

    pos_keys = [
        "COIN_COLLECTED",
        "KILLED_OPPONENT",
        "COIN_FOUND",
        "CRATE_DESTROYED",
        "CHASING_COIN",
        "CHASING_CRATE",
        "GOOD_BOMB_PLACEMENT",
        "MOVED_FROM_DANGER",
        "MOVED_FROM_BOMB",
        "CHASING_COIN_INSTEAD_OF_CRATE",
    ]

    for key, value in zip(pos_keys, values[: len(pos_keys)]):
        reward_dict[key] = float(value)

    neg_keys = [
        "WAITED",
        "INVALID_ACTION",
        "KILLED_SELF",
        "MOVED_TOWARDS_BOMB",
        "BAD_BOMB_PLACEMENT",
        "MOVED_IN_DANGER",
        "CHASING_CRATE_INSTEAD_OF_COIN",
    ]

    for key, value in zip(
        neg_keys, values[len(pos_keys) : len(pos_keys) + len(neg_keys)]
    ):
        if key == "KILLED_SELF":
            reward_dict[key] = -float(value) * 1.5
        elif key == "INVALID_ACTION":
            reward_dict[key] = -float(value) * 1.3

        else:
            reward_dict[key] = -float(value)

    movement_keys = [
        "MOVED_DOWN",
        "MOVED_LEFT",
        "MOVED_RIGHT",
        "MOVED_UP",
    ]

    reward_dict["epsilon"] = float(values[-4]) / init_range_high
    reward_dict["alpha"] = float(values[-3]) / init_range_high
    reward_dict["gamma"] = float(values[-2]) / init_range_high

    for key in movement_keys:
        reward_dict[key] = -float(values[-1])

    return reward_dict


def find_model(values, value_idx):
    return run_model(values, value_idx)


def run_model(values, value_idx, keep_model=False):

    reward_dict = make_example_dict(values)
    with open("agent_code/cc_agent_gwydion_Tuning/rewards.json", "w") as fp:
        json.dump(reward_dict, fp)

    os.system(
        "python main.py play --agent cc_agent_gwydion_Tuning --scenario coin-heaven --train 1 --no-gui --n-rounds 20"
    )

    with open("agent_code/cc_agent_gwydion_Tuning/stat.json", "r") as f:
        stats_dict = json.load(f)

    if keep_model == False:
        os.remove("agent_code/cc_agent_gwydion_Tuning/model_dict.json")
        os.remove("agent_code/cc_agent_gwydion_Tuning/my-saved-model.pt")

    return stats_dict[str(list(reward_dict.values()))]


def print_generation(gen):
    print(gen.best_solution())

In [ ]:
filename = "agent_code/cc_agent_gwydion_Tuning/genetic"

loaded_ga_instance = pygad.load(filename=filename)

In [ ]:
loaded_ga_instance.plot_fitness(ylabel="average coins")

In [ ]:
loaded_ga_instance.plot_genes(graph_type="boxplot")